In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [17]:
# read chicago housing data and convert it into a dataframe

Chicago_Housing_Data = "Resources/Chicago_Housing_Data.csv"
chicago_housing_df = pd.read_csv(Chicago_Housing_Data)
chicago_housing_df.head()

,Zipcode,Median Rent,Median Home Value,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Poverty Rate,Lat,Lng
0,60601,1944,391000.0,13887,37.6,110215,95152,1171,8.432347,41.883993,-87.619706
1,60602,1666,NaN,1277,31.1,115774,77279,32,2.505873,41.883473,-87.629127
2,60603,1630,NaN,1197,26.7,140114,106207,261,21.804511,41.880916,-87.625425
3,60604,1835,NaN,668,40.6,114565,130966,127,19.011976,41.878485,-87.627893
4,60605,1865,284200.0,26188,33.6,107811,81144,2312,8.828471,41.870331,-87.623574


In [28]:
# read yelp data for amenities and convert it into a dataframe

Yelp_Data = "Resources/Yelp_Data.csv"
yelp_df = pd.read_csv(Yelp_Data)
yelp_df.head()

,Zipcode,Total Nightlife,Total Restaurant,Total Grocery
0,60601,48,150,7
1,60602,11,78,0
2,60603,22,96,1
3,60604,5,41,0
4,60605,34,112,7


### Transform Chicago Housing DataFrame

In [18]:
# Select only the required columns and rename the columns

chicago_housing_df = chicago_housing_df[['Zipcode', 'Median Rent']]
cleaned_chicago_housing_df = chicago_housing_df.rename(columns={
    'Zipcode': 'zipcode',
    'Median Rent': 'median_rent'
})


cleaned_chicago_housing_df.head()

,zipcode,median_rent
0,60601,1944
1,60602,1666
2,60603,1630
3,60604,1835
4,60605,1865


### Transform Yelp DataFrame

In [29]:
# Select only the required columns and rename the columns

yelp_df['total_amenities'] = yelp_df['Total Nightlife'] + yelp_df['Total Restaurant'] + yelp_df['Total Grocery']

cleaned_yelp_df = yelp_df[['Zipcode', 'total_amenities']]

cleaned_yelp_df = cleaned_yelp_df.rename(columns={
    'Zipcode': 'zipcode'
})

cleaned_yelp_df.head()

,zipcode,total_amenities
0,60601,205
1,60602,89
2,60603,119
3,60604,46
4,60605,153


### Create database connection

In [30]:
# import config file to read the postgres password

from config import password 

In [31]:
# Connect to local database

connection_string = f'postgres:{password}@localhost:5432/chicago_db'
engine = create_engine(f'postgresql://{connection_string}')

In [32]:
# Confirm tables
engine.table_names()

['rent', 'amenities']

### Load DataFrames into database

In [33]:
cleaned_chicago_housing_df.to_sql(name='rent', con=engine, if_exists='append', index=False)

In [34]:
cleaned_yelp_df.to_sql(name='amenities', con=engine, if_exists='append', index=False)